### Import different library

link:-https://www.kaggle.com/c/digit-recognizer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


### Load the file and Assign the label to Y

In [2]:
train = pd.read_csv(r'/home/prajwal/Downloads/Digit/train.csv')
test = pd.read_csv(r'/home/prajwal/Downloads/Digit/test.csv')
Y_train=train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 
Y_train = to_categorical(Y_train, num_classes = 10)

Check for null values

In [3]:
X_train.isnull().any().describe()
test.isnull().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

Normalization of data

In [4]:
X_train = X_train / 255
test = test / 255

Reshape the input and split the train and test

In [5]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

4 layers of our model.The Conv2D layers followed by max pool layers are considered to be a single layer.

In [6]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

Instructions for updating:
Colocations handled automatically by placer.


Dropout is a regularization technique for reducing overfitting.
We add softmax activation unit in the end for a softmax output.

In [7]:
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Set RMSprop as optimizer

In [8]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
epochs = 30
batch_size = 64
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

 Fit the model.I didn't run all epoch due to time.

In [9]:
final = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, Y_val),callbacks=[learning_rate_reduction])
results = model.predict(test)

Instructions for updating:
Use tf.cast instead.
Train on 37800 samples, validate on 4200 samples
Epoch 1/30
37800/37800 [==============================] - 231s 6ms/step - loss: 0.2062 - acc: 0.9350 - val_loss: 0.0801 - val_acc: 0.9800
Epoch 2/30
 2240/37800 [>.............................] - ETA: 3:58 - loss: 0.0769 - acc: 0.9763

KeyboardInterrupt: 

In [10]:
results = model.predict(test)

In [11]:
results = np.argmax(results,axis = 1)

In [12]:
results = pd.Series(results,name="Label")

In [13]:
len(results)

28000

In [14]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

In [15]:
submission.to_csv("SM.csv",index=False)